# **Part I: EDA**


## **EDA Task**

### **1.** Create DataFrame "payment_enriched" (Merge "payment_report.csv" with "product.csv")

In [ ]:
# Import Google Drive files

from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/DAC_K27/Module 3 - Python/Project 2/'

# Import the "pandas" library
import pandas as pd

# Load the CSV files
payment_report = pd.read_csv(path + 'payment_report.csv', encoding = 'utf-8')
product = pd.read_csv(path + 'product.csv', encoding = 'utf-8')
transactions = pd.read_csv(path + 'transactions.csv', encoding = 'utf-8')

# Display the first few rows of each dataframe to understand their structure
payment_report.head()
len(payment_report) # 919

product.head()
len(product) # 492

transactions.head()
len(transactions) # 1324002

In [ ]:
# Check if there is any duplicate in "product_id" column in "product" DataFrame as it is the primary key
no_duplicate_id = product['product_id'].duplicated().sum()
print(f"Number of duplicate in product_id is: {no_duplicate_id}") ## 0 duplicate


In [ ]:
# Merge the "payment_report.csv" with "product.csv" using 'product_id'
payment_enriched = pd.merge(payment_report, product, on = 'product_id', how = 'left') # 'payment_report' has higher number of rows => left join

# Display the first few rows of the resulting DataFrame
payment_enriched.head()

### **2.** Explanatory Data Analysis (EDA)

#### ***2.1.*** Check each column: missing data? duplicates? incorrect data types?

In [ ]:
# 1. Checking for missing data
print("Missing data in 'payment_enriched':")
print(payment_enriched.isnull().sum()) # 'category' and 'team_own' has 22 missing values each

print("\nMissing data in 'transactions':")
print(transactions.isnull().sum()) # missing data in 'sender_id', 'receiver_id', and 'extra_info'

**Notes:**

- Missing data:
  - DataFrame '**payment_enriched**': Column 'category' and 'team_own': 22 missing values each
  - DataFrame '**transactions**': column 'sender_id': 49059; column
'receiver_id': 164795; and column 'extra_info': 1317907

- Next step: **No action** as removing those missing values might just lead to losing data.

In [ ]:
# 2. Checking for duplicates
print("Duplicate rows in 'payment_enriched':")
print(payment_enriched.duplicated().sum()) # no duplicate

print("\nDuplicate rows in 'transactions':")
print(transactions.duplicated().sum()) # 28 duplicates

**Notes:**

- Duplicate rows in '**payment_enriched**': 0
- Duplicate rows in '**transactions**': 28
- Next step: Remove those **28** duplicates in '**transactions**'.

In [ ]:
# Remove duplicate rows in 'transactions'
transactions = transactions.drop_duplicates()
print("\nDuplicate rows in 'transactions':")
print(transactions.duplicated().sum())

In [ ]:
# 3. Checking data types
print("Data types in payment_enriched:")
print(payment_enriched.dtypes)
print("\nData types in transactions:")
print(transactions.dtypes)

**Notes:**

- Data type of columns in '**payment_enriched**' has 'report_month' column whose the data type should be 'datetime' instead of 'object'
- Data type of some columns in '**transactions**' has some issues as follows.
  - '**sender_id**' and '**receiver_id**' should be in '**int64**' instead of '**float64**'
  - '**timeStamp**' should be in '**datetime**' instead of '**int64**'
- Next steps:
  - For '**payment_enriched**': Convert the datatype of '**report_month**' to '**datetime**'
  - For '**transactions**': Convert the datatype of '**sender_id**', '**receiver_id**', and '**timeStamp**' to the correct datatypes.

In [ ]:
# Convert 'report_month' to datetime format
payment_enriched['report_month'] = pd.to_datetime(payment_enriched['report_month'], format='%Y-%m')

# Convert 'sender_id' and 'receiver_id' to nullable Int64 dtype ('sender_id' and 'receiver_id' include NaN so we cannot use 'int64')
transactions['sender_id'] = transactions['sender_id'].astype('Int64')
transactions['receiver_id'] = transactions['receiver_id'].astype('Int64')

# Convert 'timeStamp' to 'datetime'
transactions['timeStamp'] = pd.to_datetime(transactions['timeStamp'], unit='ms')

# Verify the changes
print(payment_enriched.dtypes)
payment_enriched.head()

print(transactions.dtypes)
transactions.head()

#### ***2.2.*** Summarize numerical data: any incorrect values?

In [ ]:
# Summarize numerical data
payment_enriched.describe() # All good

transactions.describe() # negative values in 'transStatus', 'receiver_id'

In [ ]:
# Histogram of numerical columns
import matplotlib.pyplot as plt

# Generate histograms for all numerical columns in 'payment_enriched'
payment_enriched.hist(figsize=(10, 8), bins=20, edgecolor='black')
plt.suptitle('Histograms of Numerical Columns in "payment_enriched" DataFrame')
plt.show()

In [ ]:
# Generate histograms for all numerical columns in transactions
transactions.hist(figsize=(10, 8), bins=20, edgecolor='black')
plt.suptitle('Histograms of Numerical Columns in "transactions" DataFrame')
plt.show()

**Notes:**

- For '**payment_enriched**': numerical data looks all good
- For '**transactions**': there are negatives values in '**transStatus**' and '**receiver_id**' => Next steps: double check with relevant department to fix incorrect ids. However, this is minor mistake.

# **Part II: Data Wrangling**

##1. Using '**payment_report.csv**' & '**product.csv**' (i.e., the DataFrame '**payment_enriched**'), identify top 3 product_ids with the highest volume

In [ ]:
# Identify the top 3 product_ids with the highest volume
top_3_products = payment_enriched.groupby('product_id')['volume'].sum().nlargest(3)

print("Top 3 Product IDs with the Highest Volume:")
top_3_products.reset_index()

## 2.	Given that each 'product_id' is only owed by 1 team, check if there is any abnormal product that is against this rule?

In [ ]:
# Group by 'product_id' and check the number of unique teams
abnormal_products = payment_enriched.groupby('product_id')['team_own'].nunique()

# Filter out product_ids with more than one unique team
abnormal_products = abnormal_products[abnormal_products != 1]

print("Abnormal product_ids found:")
abnormal_products.reset_index()['product_id']

## 3.	Find the team has had the lowest performance (lowest 'volume') since Q2.2023. Find the category that contributes the least to that team.

In [ ]:
# Filter data for Q2 2023 onward
q2_2023_onward = payment_enriched[payment_enriched['report_month'] >= '2023-04-01']

# Group by 'team_own' and sum the volume to find the lowest performing team
team_performance = q2_2023_onward.groupby('team_own')['volume'].sum()
lowest_team = team_performance.idxmin()

print(f"Team with the lowest performance since Q2 2023: {lowest_team}")

# Filter the data for the lowest performing team
lowest_team_data = q2_2023_onward[q2_2023_onward['team_own'] == lowest_team]

# Group by 'category' and sum the volume to find the least contributing category
category_performance = lowest_team_data.groupby('category')['volume'].sum()
lowest_category = category_performance.idxmin()

print(f"Category contributing the least to team {lowest_team}: {lowest_category}")


## 4.	Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?

In [ ]:
# Filter data for refund transactions (payment_group = ‘refund’) only
refund_data = payment_enriched[payment_enriched['payment_group'] == 'refund']

# List the source_ids contributing to refund transactions
refund_source_id = refund_data.groupby('source_id')['volume'].sum() # group by 'source_id' and sum the volume
print("Source_ids contributing to refund transactions: ")
refund_source_id.reset_index()['source_id']

In [ ]:
# Identify the 'source_id' with the highest contribution
highest_source_id = refund_source_id.idxmax()

print(f"The 'source_id' with the highest contribution: {highest_source_id}")

## 5. Using 'transactions.csv', define type of transactions (‘transaction_type’) for each row, given:
- transType = 2 & merchant_id = 1205: Bank Transfer Transaction
- transType = 2 & merchant_id = 2260: Withdraw Money Transaction
- transType = 2 & merchant_id = 2270: Top Up Money Transaction
- transType = 2 & others merchant_id: Payment Transaction
- transType = 8, merchant_id = 2250: Transfer Money Transaction
- transType = 8 & others merchant_id: Split Bill Transaction
- Remained cases are invalid transactions

In [ ]:
# Define the function (underlying conditions to categorize the transaction types)
def defined_transaction_type(row):
    if row['transType'] == 2 and row['merchant_id'] == 1205:
        return 'Bank Transfer Transaction'
    elif row['transType'] == 2 and row['merchant_id'] == 2260:
        return 'Withdraw Money Transaction'
    elif row['transType'] == 2 and row['merchant_id'] == 2270:
        return 'Top Up Money Transaction'
    elif row['transType'] == 2:
        return 'Payment Transaction'
    elif row['transType'] == 8 and row['merchant_id'] == 2250:
        return 'Transfer Money Transaction'
    elif row['transType'] == 8:
        return 'Split Bill Transaction'
    else:
        return 'Invalid Transaction'

# Create a new column named 'transaction_type'
transactions['transaction_type'] = transactions.apply(defined_transaction_type, axis=1)

# Display the updated DataFrame
transactions[['transaction_id', 'merchant_id', 'transType', 'transaction_type']].head()
transactions


## 6.	Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers

In [ ]:
# @title
# Exclude invalid transactions
valid_transactions = transactions[transactions['transaction_type'] != 'Invalid Transaction']

# Group by 'transaction_type' and calculate the metrics
summary = valid_transactions.groupby('transaction_type').agg(
    no_transactions=('transaction_id', 'count'),
    total_volume=('volume', 'sum'),
    unique_senders=('sender_id', 'nunique'),
    unique_receivers=('receiver_id', 'nunique')
).reset_index()

# Display the summary DataFrame
summary


# **This is the end of Project 2 - Thank you very much for your time!** 😀